In [ ]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

edges = [[75582971913961693, 75582971913961503]]
print(cg.get_cross_chunk_edges_layer(edges))
print(cg.get_parents(edges[0]))

In [4]:
edges = cg.read_chunk_edges([cg.get_chunk_id(layer=2,x=50,y=100,z=0)])
edges = edges["between"].get_pairs()
nodes = np.unique(edges)
parents = cg.get_roots(nodes)
d = dict(zip(parents, nodes))
print(d)

{864691138338302132: 75582971913961693, 864691137629078617: 75512534450307707, 864691137995403800: 75582903194484819, 864691137995404568: 75582834475009044, 864691138741942725: 75582971913961542, 864691137995404824: 75582903194484963, 864691136334203586: 75582971913961503}


In [9]:
parents = np.array(edges[0][0])
while parents.size:
    parents = cg.get_parent(parents)
    print(parents)

147640565951889409
219704963218014230
291635563662936192
363718446468361184
435828817064694903
506866064312035921
576742227280767262
864691138338302132
None


AttributeError: 'NoneType' object has no attribute 'size'

In [2]:
from pychunkedgraph.graph.edits import add_edge
from pychunkedgraph.graph.utils.context_managers import TimeIt

edges = np.array(edges, dtype=np.uint64)
with TimeIt("add_edge"):
    print(add_edge(cg, edges=edges))

get_cross_chunk_edges cross_edges_d 0.20497512817382812
add_edge 0.4112589359283447


ValueError: could not broadcast input array from shape (11) into shape (10)